In [24]:
import subprocess

# Activate the Conda environment
subprocess.run(["conda", "activate", "arcgispro-py3-clone"], shell=True)

CompletedProcess(args=['conda', 'activate', 'arcgispro-py3-clone'], returncode=1)

In [21]:
import geopandas as gp
import pandas as pd
from shapely.geometry import Polygon
from routingpy.routers import MapboxOSRM
import numpy as np

mb = MapboxOSRM(api_key = "YOU_KEY_HERE")

Stations = gp.read_file(r"path\\to\\EVStations.zip")

#calculate isochrones using drive time of 6, 8, and 10 min
def mb_isochrone(station, time=[6, 8, 10], profile="driving"):
    # Grab X and Y values in 4326
    station['LON_VALUE'] = station.to_crs(4326).geometry.x
    station['LAT_VALUE'] = station.to_crs(4326).geometry.y

    coordinates = station[['LON_VALUE', 'LAT_VALUE']].values.tolist()

    # Build a list of shapes
    isochrone_shapes = []

    if type(time) is not list:
        time = [time]

    # Use minutes as input, but the API requires seconds
    time_seconds = [60 * x for x in time]

    for c in coordinates:
        iso_request = mb.isochrones(locations=c, profile=profile,
                                    intervals=time_seconds, polygons="true")

        for i in iso_request:
            iso_geom = Polygon(i.geometry[0])
            isochrone_shapes.append(iso_geom)

    # Re-build the dataset with isochrone geometries
    df_values = station.drop(columns=['geometry', 'LON_VALUE', 'LAT_VALUE'])

    time_col = time * len(df_values)

    # Repeat the dataframe to account for multiple time intervals
    df_values_rep = pd.DataFrame(np.repeat(df_values.values, len(time_seconds), axis=0))
    df_values_rep.columns = df_values.columns

    isochrone_gdf = gp.GeoDataFrame(
        data=df_values_rep,
        geometry=isochrone_shapes,
        crs=4326
    )

    isochrone_gdf['time'] = time_col

    # Sort the dataframe in descending order of time
    isochrone_gdf = isochrone_gdf.sort_values('time', ascending=False)

    return isochrone_gdf

# Process each station individually
for idx, station in Stations.iterrows():
    station_gdf = gp.GeoDataFrame([station], geometry=[station.geometry], crs=Stations.crs)
    isochrones = mb_isochrone(station_gdf, time=[6, 8, 10], profile="driving")
    
    # Save each isochrone individually
    output_file = f"G:\\service_area_station_{idx}.shp"
    isochrones.to_file(output_file, driver="ESRI Shapefile")
    print(f"Saved service area for station {idx} to {output_file}")


Saved service area for station 0 to G:\service_area_station_0.shp
Saved service area for station 1 to G:\service_area_station_1.shp
Saved service area for station 2 to G:\service_area_station_2.shp
Saved service area for station 3 to G:\service_area_station_3.shp
Saved service area for station 4 to G:\service_area_station_4.shp
Saved service area for station 5 to G:\service_area_station_5.shp
Saved service area for station 6 to G:\service_area_station_6.shp
Saved service area for station 7 to G:\service_area_station_7.shp
Saved service area for station 8 to G:\service_area_station_8.shp


In [23]:
isochrones.explore(column = "time")
